## Tutorial to use histone mark age predictors

In [3]:
# Load required packages
import pandas as pd
import joblib
import anndata
import pyBigWig as pbw
import numpy as np
from tqdm.notebook import tqdm
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import ARDRegression, ElasticNet

In [4]:
# Download an example from the ENCODE project (keep in mind this is a training sample used in the models)
!wget https://www.encodeproject.org/files/ENCFF386QWG/@@download/ENCFF386QWG.bigWig --quiet

In [7]:
def process_bigWig(bigWig_file_path, annotation_file_path='../metadata/Ensembl-105-EnsDb-for-Homo-sapiens-genes.csv'):
    """
    Process the given bigWig file to extract genomic annotations and transform signal values.

    Parameters:
        bigWig_file_path (str): Path to the bigWig file containing signal values.
        annotation_file_path (str): Path to the CSV file containing genomic annotations. Default value points to a specific dataset.

    Returns:
        pandas.DataFrame: A DataFrame containing transformed signal values for each gene.
    """
    # Get genomic annotation
    chromosomes = ['1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '3', '4', '5', '6', '7', '8', '9', 'X']
    genes = pd.read_csv(annotation_file_path)
    genes = genes[genes['chr'].apply(lambda x: x in chromosomes)]
    genes.index = genes.gene_id

    # Read bigWig p-value file (in the ENCODE project, bigWig p-values are already log-transformed)
    bw = pbw.open(bigWig_file_path)

    # Transform into a table of transformed signal values for each gene
    signal_sample = np.empty(shape=(0, 0), dtype=float)
    print('Processing file...')
    for i in tqdm(range(genes.shape[0])):
        try: 
            signal = bw.stats('chr' + genes['chr'].iloc[i], genes['start'].iloc[i] - 1, genes['end'].iloc[i], type='mean', exact=True)[0]
        except:
            signal = None
        if signal is not None:
            signal_transformed = np.arcsinh(signal)
        else:
            signal_transformed = 0
        signal_sample = np.append(signal_sample, signal_transformed)
    print('Done!')

    sample = pd.DataFrame(signal_sample[None, :], columns=genes.gene_id.tolist())

    return sample


def predict_histone_mark_age(processed_sample, histone):
    """
    Predict age based on the processed sample using a trained model for a given histone type.

    Parameters:
        processed_sample (pandas.DataFrame): Processed sample containing transformed signal values for genes.
        histone (str): Histone type for which the age prediction model is trained.

    Returns:
        numpy.ndarray: Predicted age for the given processed sample.
    """
    feature_selector_path = '../results/models/' + histone + '_feature_selector.pkl'
    feature_selector = joblib.load(feature_selector_path)

    dim_reduction_path = '../results/models/' + histone + '_dim_reduction.pkl'
    dim_reduction = joblib.load(dim_reduction_path)

    model_path = '../results/models/' + histone + '_model.pkl'
    model = joblib.load(model_path)

    selected_features = processed_sample.loc[:, np.abs(feature_selector.coef_) > 0]
    processed_sample_reduced = dim_reduction.transform(selected_features)
    y_hat = model.predict(processed_sample_reduced)

    return y_hat

In [11]:
#use your sample path
sample = process_bigWig('ENCFF386QWG.bigWig')
#choose histone mark from H3K4me3, H3K9me3, H3K27me3, H3K36me3, H3K4me1, H3K9ac, H3K27ac, and pan_histone
histone_mark = 'H3K4me3'
#choose from the available histone marks
y_hat = predict_histone_mark_age(sample, histone=histone_mark)[0]

Processing file...


  0%|          | 0/62241 [00:00<?, ?it/s]

Done!
The predicted H3K4me3 age is 53.999 years.


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator ElasticNet from version 1.0.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator TruncatedSVD from version 1.0.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator ARDRegression from version 1.0.1 when using version 1.2.2. This mi

In [12]:
print(f'The predicted {histone_mark} age is {round(y_hat,3)} years.')

The predicted H3K4me3 age is 53.999 years.
